# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770912216008                   -0.52    9.0         
  2   -2.772149607843       -2.91       -1.32    1.0    136ms
  3   -2.772170324301       -4.68       -2.45    1.0    165ms
  4   -2.772170656526       -6.48       -3.17    1.0    140ms
  5   -2.772170722766       -7.18       -4.21    2.0    171ms
  6   -2.772170722892       -9.90       -4.39    1.0    161ms
  7   -2.772170723010       -9.93       -5.14    1.0    145ms
  8   -2.772170723015      -11.34       -5.67    1.0    161ms
  9   -2.772170723015      -12.27       -6.54    2.0    169ms
 10   -2.772170723015      -14.65       -6.76    1.0    165ms
 11   -2.772170723015      -14.21       -7.74    1.0    161ms
 12   -2.772170723015      -15.35       -8.24    1.0    171ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.7

1.7735582742911242

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770759656811                   -0.53    9.0         
  2   -2.772054707116       -2.89       -1.32    1.0    170ms
  3   -2.772083109650       -4.55       -2.53    1.0    139ms
  4   -2.772083383775       -6.56       -3.48    1.0    155ms
  5   -2.772083416429       -7.49       -3.94    2.0    191ms
  6   -2.772083417778       -8.87       -5.47    1.0    143ms
  7   -2.772083417810      -10.49       -5.71    3.0    194ms
  8   -2.772083417811      -12.48       -5.90    1.0    149ms
  9   -2.772083417811      -13.09       -6.93    1.0    166ms
 10   -2.772083417811      -14.40       -7.25    1.0    157ms
 11   -2.772083417811      -13.85       -7.98    1.0    163ms
 12   -2.772083417811      -14.57       -8.32    2.0    178ms

Polarizability via ForwardDiff:       1.7725349810579116
Polarizability via finite difference: 1.7735582742911242
